In [178]:
import numpy as np
import os
import sys
import re
from functools import reduce
from collections import defaultdict, OrderedDict
import itertools
import time
from collections import deque
from decimal import Decimal


In [208]:
query_file = 'queries.txt'


class Timer:
    """With block timer.
    with Timer() as t:
        foo = blah()
    print('Request took %.03f sec.' % t.interval)
    """
    def __enter__(self):
            self.start = time.perf_counter()
            return self

    def __exit__(self, *args):
            self.end = time.perf_counter()
            self.interval = self.end - self.start
            


class OrderedDefaultDict(OrderedDict):
    factory = list

    def __missing__(self, key):
        self[key] = value = self.factory()
        return value

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

class Parser:
    def __init__(self):
        pass
    
    @classmethod
    def remove_comments(cls,s):
        if '//' in s:
            s = s[:s.find('//')] #Remove comments
        s = s.strip()
        return s
    @classmethod
    def splitlr(cls,s):
        left,right = s.split(':=')
        left = left.strip()
        right = right.strip()
        return left,right

    @classmethod
    def remove_outer_paren(cls,s):
        s = s[s.find('(')+1:s.find(')')]
        return s
    
    @classmethod
    def parse(cls,s,call):
        if call == 'input':
            return cls.remove_outer_paren(s)
        elif call == 'select':
            s = s[s.find("(")+1:s.rfind(")")]
            base_table,args = s.split(',')
            base_table = base_table.strip()
            args = args.strip()
            
            if 'or' not in args and 'and' not in args:
                return base_table,args,None
            condition = 'or' if 'or' in args else 'and'
            
            args = args.split(condition)
            args = [cls.remove_outer_paren(s.strip()) for s in args]
            return base_table,args,condition
        elif call == 'project':
            s = s[s.find("(")+1:s.rfind(")")]
            base_table,args = s.split(',')[0],s.split(',')[1:]
            base_table = base_table.strip()
            args = [a.strip() for a in args]
            
            return base_table,args
        
        elif call == 'avg':
            s = s[s.find("(")+1:s.rfind(")")]
            base_table,args = s.split(',')[0],s.split(',')[1]
            base_table = base_table.strip()
            args = args.strip()
            return base_table,args
        
        
        elif call == 'join':
            s = s[s.find("(")+1:s.rfind(")")]
            t1,t2,args = s.split(',')[0],s.split(',')[1],s.split(',')[2:] 
            t1 = t1.strip()
            t2 = t2.strip()
            args = [a.strip() for a in args]
            
            
            if ' and ' not in args[0]:
                return t1,t2,args[0].strip(),None
            
            condition = 'and'
            args = ','.join(args).split(condition)
            args = [cls.remove_outer_paren(s.strip()) for s in args]
            return t1,t2,args,condition
            
    @classmethod
    def npy_to_dict(cls,arr):
        D = AttrDict()
        r,c = arr.shape
        keys = arr[0]
        for i,k in enumerate(keys):
            try:
                D[k] = arr[1:,i].astype(int)
            except:
                D[k] = arr[1:,i]
        return D

class Table:
    def __init__(self,table_name,inp_name = None, t = None):
        self.name = table_name
        if inp_name:
            arr = np.loadtxt(f"{inp_name}.txt",dtype=str,delimiter = "|")
            self.t = Parser.npy_to_dict(arr)
        else:
            self.t = t
    
    def select(self,args,condition,name):
        if condition:
            args = ['self.t.' + arg for arg in args]
            indices = []
            for a in args:
                i = eval(f'np.where({a})')
                indices.append(i)
                
            if condition == 'or':
                indices = reduce(np.union1d, indices)
            else:
                indices = reduce(np.intersect1d, indices)
            
        else:
            args = args.replace('=','==')
            args = 'self.t.' + args
            indices = eval(f'np.where({args})')

        new_table = {k:v[indices] for k,v in self.t.items()}
        new_table = AttrDict(new_table)
        
        new_table = Table(name, t = new_table)
        return new_table
    
    def project(self,args,name):
        new_table = {k:self.t[k] for k in args}
        new_table = AttrDict(new_table)
        
        new_table = Table(name, t = new_table)
        return new_table
        
    def avg_sum(self,args,operation = 'avg'):
        
        arr = self.t[args]
        if operation == 'sum':
            ans = np.sum(arr)
        else:
            ans = np.average(arr)
        return str(ans)
    
    def avg_sum_group(self,args,name,operation = 'avg'):
        ar1 = args[0]
        ar_group = args[1:]
        
        groups = [self.t[arg] for arg in ar_group]
        
        groups = list(zip(*groups))
        
        d = defaultdict(list)
        for i,g in enumerate(groups):
            d[g].append(i)
        
        for k,ind in d.items():
            if operation == 'avg':
                d[k] = np.mean(self.t[ar1][ind])
            else:
                d[k] = np.sum(self.t[ar1][ind])
        
        d = OrderedDict(sorted(d.items()))
        
        new_table = defaultdict(list)
        new_table[ar1] = np.array(list(d.values()))
        
        for key_tuple in d.keys():
            for arg, key in zip(ar_group, key_tuple):
                new_table[arg].append(key)
                
        new_table = Table(name, t = new_table)
        return new_table
    
    def sort(self,args,name):
        s = ','.join('self.t.'+a for a in args)
        to_sort = eval(f'list(zip({s}))')
        indices = Table.argsort(to_sort)
        
        new_table = {k:v[indices] for k,v in self.t.items()}
        new_table = Table(name, t = new_table)
        return new_table
    
    def moving_avg_sum(self,args,name,operation = 'avg'):
        col,N = args
        
        arr = self.t[col]
        
        agg = Table.moving_agg(arr,int(N),operation)
        
        col_name = f'mov_{operation}'
        
        new_table = {}
        
        new_table[col] = arr
        new_table[col_name] = agg
        new_table = Table(name, t = new_table)
        return new_table
    
    @staticmethod
    def argsort(lis):
        return sorted(range(len(lis)), key=lis.__getitem__)

    
    @staticmethod
    def find_operator(s):
        lis = ['==','<=','>=','<','>','!=','=']
        for l in lis:
            if l in s:
                return l
    
    @staticmethod
    def moving_agg(x, N,operation = 'avg'):
        ar = deque([])
        res = []
        for n in x:
            ar.append(n)
            if len(ar) == N+1:
                ar.popleft()
            if operation == 'avg':
                res.append("{0:.4f}".format(np.mean(ar)))
            else:
                res.append(np.sum(ar))
        return res
    
    
    
    @staticmethod
    def join(name1,name2,args,condition,name,t1,t2):
        d1,d2 = t1.t, t2.t
        
        k1,k2 = list(d1.keys()), list(d2.keys())
        n1,n2 = len(d1[k1[0]]), len(d2[k2[0]])
        indices = []
        
        exec(f'{name1} = t1.t')
        exec(f'{name2} = t2.t')
        
        if condition:
            min_len = float('inf')
            min_index = -1
            min_lis = []
            
            for i,s in enumerate(args):
                
                lhs = ''
                rhs = ''
                if Table.find_operator(s) == '=': 
                    s = s.replace('=','==')
                    
                for key in k1:
                    sub = f'{name1}.{key}'
                    if sub in s:
                        lhs = sub
                        break

                for key in k2:
                    sub = f'{name2}.{key}'
                    if sub in s:
                        rhs = sub
                        break
                
                op = Table.find_operator(s)
                s = f'xx {op} yy'
                
                xx,yy = eval(f'np.meshgrid({lhs}, {rhs}, sparse=False, indexing="ij")')
                ind1,ind2 = eval(f'np.where({s})')
                
                if len(ind1) < min_len:
                    min_len = len(ind1)
                    min_lis = [ind1,ind2]
                    min_index = i
            
            
            for i,s in enumerate(args):
                if i != min_index:
                    for key in k1:
                        sub = f'{name1}.{key}'
                        if sub in s:
                            s = s.replace(sub, sub + '[m]')
                            break

                    for key in k2:
                        sub = f'{name2}.{key}'
                        if sub in s:
                            s = s.replace(sub,sub + '[n]')
                            break
                    ind1 = []
                    ind2 = []
                    for m,n in zip(*min_lis):
                        if eval(s):
                            ind1.append(m)
                            ind2.append(n)
                    min_lis = [ind1,ind2]
                
            indices1 = np.array(min_lis[0])
            indices2 = np.array(min_lis[1])
            
        else:
            s = args
            if '=' in s and ('<' not in s and '>' not in s): 
                s = s.replace('=','==')
            
            lhs = ''
            rhs = ''


            for key in k1:
                sub = f'{name1}.{key}'
                if sub in s:
                    lhs = sub
                    break

            for key in k2:
                sub = f'{name2}.{key}'
                if sub in s:
                    rhs = sub
            
            
            op = Table.find_operator(s)
            s = f'xx {op} yy'

            xx,yy = eval(f'np.meshgrid({lhs}, {rhs}, sparse=False, indexing="ij")')
            indices1,indices2 = eval(f'np.where({s})')
            
        
        d1 = {f'{name1}_{k}':v[indices1] for k,v in d1.items()}
        d2 = {f'{name2}_{k}':v[indices2] for k,v in d2.items()}
        d_merged = OrderedDict({**d1, **d2})
        
        new_table = AttrDict(d_merged)
        new_table = Table(name, t = new_table)
        return new_table
        
    def __str__(self):
        keys = list(self.t.keys())
        s = ''
        n = len(self.t[keys[0]])
        
        max_size = [0]*len(keys)
        for i,k in enumerate(keys):
            for a in self.t[k]:
                max_size[i] = max(max_size[i], len(str(a)))
        
        
        
        for i in range(n):
            for j,k in enumerate(keys):
                ms = max_size[j]
                format_str = '{:<' + str(ms+2) + '}'
                s += format_str.format(self.t[k][i])
            s += '\n'
            
                
        return s
    def __repr__(self):
        return self.__str__()

class Solution:
    def __init__(self):
        pass
    
    def read_line(self,line):
        
        line = Parser.remove_comments(line)
        if line:
            with Timer() as t:
                if ':=' in line:
                    left,right = Parser.splitlr(line)
                    if 'inputfromfile' in right:
                        inp_name = Parser.parse(right,'input')
                        exec(f'self.{left} = Table(left,inp_name)')

                    elif 'select' in right:
                        base_table, args,condition = Parser.parse(right,'select')
                        exec(f'self.{left} = self.{base_table}.select(args,condition,left)')

                    elif 'project' in right:
                        base_table, args = Parser.parse(right,'project')
                        exec(f'self.{left} = self.{base_table}.project(args,left)')

                    elif 'avggroup' in right:
                        base_table, args = Parser.parse(right,'project')
                        exec(f'self.{left} = self.{base_table}.avg_sum_group(args,left)')

                    elif 'movavg' in right:
                        base_table, args = Parser.parse(right,'project')
                        exec(f'self.{left} = self.{base_table}.moving_avg_sum(args,left)')

                    elif 'avg' in right:
                        base_table, args = Parser.parse(right,'avg')
                        exec(f'self.{left} = self.{base_table}.avg_sum(args)')

                    elif 'sumgroup' in right:
                        base_table, args = Parser.parse(right,'project')
                        exec(f'self.{left} = self.{base_table}.avg_sum_group(args,left,"sum")')

                    elif 'movsum' in right:
                        base_table, args = Parser.parse(right,'project')
                        exec(f'self.{left} = self.{base_table}.moving_avg_sum(args,left,"sum")')

                    elif 'sum' in right:
                        base_table, args = Parser.parse(right,'avg')
                        exec(f'self.{left} = self.{base_table}.avg_sum(args,sum)')

                    elif 'sort' in right:
                        base_table, args = Parser.parse(right,'project')
                        exec(f'self.{left} = self.{base_table}.sort(args,left)')
                    elif 'join' in right:
                        t1,t2,args,condition = Parser.parse(right,'join')
                        exec(f'self.{left} = Table.join(t1,t2,args,condition,left,{"self."+t1},{"self." + t2})')
                
            print(line,'\nQuery took %.06f sec.\n' % t.interval) 
    

In [209]:
solution = Solution()
with open(query_file,'r') as f:
    for i,line in enumerate(f):
        solution.read_line(line)


R := inputfromfile(sales1) 
Query took 0.024852 sec.

R1 := select(R, (time > 50) or (qty < 30)) 
Query took 0.001121 sec.

R2 := project(R1, saleid, qty, pricerange) 
Query took 0.000079 sec.

R3 := avg(R1, qty) 
Query took 0.000130 sec.

R4 := sumgroup(R1, time, qty) 
Query took 0.001187 sec.

R5 := sumgroup(R1, qty, time, pricerange) 
Query took 0.002616 sec.

R6 := avggroup(R1, qty, pricerange) 
Query took 0.000785 sec.

S := inputfromfile(sales2) 
Query took 0.635050 sec.

T := join(R, S, R.customerid = S.C) 
Query took 0.815189 sec.

T1 := join(R1, S, (R1.qty > S.Q) and (R1.saleid = S.saleid)) 
Query took 2.042371 sec.

T2 := sort(T1, S_C) 
Query took 0.000549 sec.

T2prime := sort(T1, R1_time, S_C) 
Query took 0.000583 sec.

T3 := movavg(T2prime, R1_qty, 3) 
Query took 0.004367 sec.

T4 := movsum(T2prime, R1_qty, 5) 
Query took 0.002726 sec.

Q1 := select(R, qty = 5) 
Query took 0.000070 sec.

Btree(R, qty) 
Query took 0.000001 sec.

Q2 := select(R, qty = 5) 
Query took 0.000054

In [219]:
s = 'avg(R1, qty) '
Parser.parse(s,'avg')

('R1', 'qty')

In [221]:
a = np.array([['Name','ID', 'Qty'],['a',0,12], ['c',1,15],['e',2,13]])

In [222]:
Parser.npy_to_dict(a)

{'Name': array(['a', 'c', 'e'], dtype='<U4'),
 'ID': array([0, 1, 2]),
 'Qty': array([12, 15, 13])}

In [223]:
x = [3,2,4,1,6]

In [224]:
Table.argsort(x)

[3, 1, 0, 2, 4]

In [225]:
tables = [solution.Q2.t, solution.Q4.t]

In [226]:
tables[0].keys()

dict_keys(['saleid', 'itemid', 'customerid', 'storeid', 'time', 'qty', 'pricerange'])

In [35]:
from BTrees.OOBTree import OOBTree

In [36]:
btree = OOBTree()


In [37]:
btree[2] = 3

In [38]:
btree[3] = "345"

In [39]:
btree[4] = [1,4,6,7]

In [54]:
list(btree.values(2,3))

[3, '345']

In [62]:
bisect.bisect_left(btree.keys(),1)

0

In [63]:
list(btree.keys())

[2, 3, 4]

In [22]:
a = np.array([1,1,2,3,5,4,4,2,1,6,7,8,3,1,2])

In [58]:
import bisect